# **绘制实时音频频谱（Real-time Audio Signal Frequency Spectrum）**

**使用 Python 获取音频信号并做 FFT 分析，使用 CodeLab Adapter Linda 发送数据，使用 Scratch 实时画图**

In [2]:
import pyaudio
import numpy as np
import time

from codelab_adapter_client import AdapterNode

class MyNode(AdapterNode):
    NODE_ID = "linda/test"

    def __init__(self):
        super().__init__()
        
node = MyNode()
node.receive_loop_as_thread()



time.sleep(0.1)

buff_size = 8820                    
wanted_num_of_bins = 12              
fs = 44100
pa = pyaudio.PyAudio()
stream = pa.open(format=pyaudio.paInt16, channels=1, rate=fs, input=True, frames_per_buffer=int(buff_size))


end_index = []
for i in np.arange(12):    
    end_index.append(int((buff_size/2)/2**i))
end_index.reverse()

    
start_index = [0]
for i in end_index[:-1]:
    start_index.append(i+1)        

#n=0

while True:

    block = stream.read(int(buff_size))
    data = np.fromstring(block, dtype=np.int16)

    x = np.array(data)/(2**15)
    seg_len = len(x)

    
    X = np.abs(np.fft.fft(x))[0:int(seg_len/2)]


    X2 = []
    for i in np.arange(12):
        XX = np.mean(X[start_index[i]:end_index[i]])
        #XXlog = np.log10(XX)
        #X2.append(XXlog)
        X2.append(XX)
        i=i+1
    
    outlist = ['X2_12',X2]
    
    node.linda_out(outlist)

    # n = n + 1




<ipython-input-2-5e39c5b56580>:42: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(block, dtype=np.int16)


KeyboardInterrupt: 

In [3]:
X2

[0.7223585122766498,
 0.7333254574911671,
 1.222644784726889,
 1.7089205931297038,
 1.1150328993779373,
 0.7718098785159181,
 0.3260303176897742,
 0.12371829838066968,
 -0.12173466466699337,
 -0.33312832601198866,
 -0.7870554521258644,
 -1.547234047545305]

In [8]:
X(dtype = np.int16)

TypeError: 'numpy.ndarray' object is not callable